<a href="https://colab.research.google.com/github/leoscarlato/math-language-project/blob/main/Projeto_Linguagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NOME da linguagem: **{mat}**

POR: Alexandre Magno e Leonardo Scarlato


**{mat}** consiste em uma linguagem de programação interpretada, com a finalidade de facilitar a escrita em português de operações matemáticas. Nesta linguagem, há palavras reservadas que representam operações que proximam a dialética matemática oral com o código. Assim, proporciando um espaço mais amigavél para estudos de ambas ciencias.

O público alvo dessa linguagem são de jovens no estágio de desenvolvimento do estudo matemático e de programação.

**{mat}** lidará com números pertencentes ao conjunto dos Racionais(Q). Contudo, na linguagem, trabalha-se com a separação entre inteiros (int) e fracionarios (float).

In [2]:
!pip install rply

In [6]:
# Analisador Lexico

from rply import LexerGenerator

lg = LexerGenerator()

lg.add("NUMBER", r"\d+")
lg.add("PLUS", r"\+")
lg.add("MINUS", r"\-")
lg.add("MUL", r"\*")
lg.add("DIV", r"/")
lg.add("ROOT", r"raiz_de")
lg.add("POW", r"elevado_a")
lg.add("FACT", r"fatorial")
lg.add("PI", r"pi")
lg.add("SIN", r"seno")
lg.add("COS", r"cosseno")
lg.add("TAN", r"tangente")

lg.add("INT", r"int")
lg.add("FLOAT", r"float")

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')

lg.add('EQUALS', r"=")

lg.ignore('\s+')

lexer = lg.build()

In [ ]:
# Analisador Sintatico

# TODO Inserir classes ifElse, While

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self,decls,stmts):
        self.decls = decls
        self.stmts = stmts
    
    def accept(self, visitor):
        visitor.visit_prog(self)

class Declatations(BaseBox):
    def __init__(self,decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_declarations(self)

class Declaration(BaseBox):
    def __init__(self,id,typee):
        self.id = id
        self.typee = typee
        
    def accept(self, visitor):
        visitor.visit_declaration(self)

class Statements(BaseBox):
    def __init__(self,stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self,id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value

class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
    pass

class Sub(BinaryOp):
    pass

class Mul(BinaryOp):
    pass

class Div(BinaryOp):
    pass

class Pow(BinaryOp):
    pass

class Root(BinaryOp):
    pass

class Fact(BinaryOp):
    pass

class Sin(BinaryOp):
    pass

class Cos(BinaryOp):
    pass

class Tan(BinaryOp):
    pass